# Introducción

En este notebook se procesarán los videos y se extraerán las caracteristicas de los clips usando el modelo preentrenado X3D


In [2]:
import torch
import torchvision.transforms as transforms
import torchvision.models.video as models
import cv2
import numpy as np
from PIL import Image
from imblearn.over_sampling import SMOTE
import re
import os
from collections import defaultdict
import json
import pickle
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [3]:
# Funcion para agrupar la ruta de los videos por accion   y ordenar el diccionario según la acción

def group_videos_by_action(root_path, video_extensions=None):

    if video_extensions is None:
        video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.flv', '.wmv']

    grouped_videos = defaultdict(list)

    for dirpath, dirnames, filenames in os.walk(root_path):
        for file in filenames:
            if any(file.lower().endswith(ext) for ext in video_extensions):
                action_name = os.path.basename(dirpath)  
                file_path = os.path.join(dirpath, file)
                grouped_videos[action_name].append(file_path)
    
    sorted_actions = sorted(
        grouped_videos.items(), 
        key=lambda x: int(x[0].split('_')[1])  # Extraer número después de "action_"
    )
    return grouped_videos



def ordenar_diccionario(input_dict):
    # Ordenamos las claves extrayendo el número
    sorted_keys = sorted(input_dict.keys(), key=lambda x: int(re.search(r'\d+', x).group()))
    
    # Creamos un nuevo defaultdict ordenado
    sorted_dict = defaultdict(list, {key: input_dict[key] for key in sorted_keys})

    return sorted_dict

root_path = "F:/data/mvfouls/augmented_videos_severity"
videos_path = group_videos_by_action(root_path = root_path)
sorted_keys = ordenar_diccionario(videos_path)


In [ ]:
#Dividimos las acciones en grupos para facilitar su procesamiento
def save_dict_chunks(input_dict, chunk_size, save_path, format='npy'):

    keys = list(input_dict.keys())
    for i in range(0, len(keys), chunk_size):
        chunk = {key: input_dict[key] for key in keys[i:i + chunk_size]}
        file_name = f"{save_path}_chunk_{i // chunk_size}.{format}"
        
        if format == 'npy':
            np.save(file_name, chunk)
        elif format == 'pkl':
            with open(file_name, 'wb') as f:
                pickle.dump(chunk, f)
        else:
            raise ValueError("Formato no soportado. Usa 'npy' o 'pkl'.")
        print(f"Guardado: {file_name}")



save_dict_chunks(
    sorted_keys,
    chunk_size=500,  # Número de elementos por archivo
    save_path='F:/data/mvfouls/train_augmented',
    format='npy'  
)


In [ ]:
#Cargamos el chunk a procesar
chunk = np.load('F:/data/mvfouls/train_augmented_chunk_0.npy', allow_pickle=True).item()


In [ ]:

# Cargar el modelo MViTv2 preentrenado
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.mvit_v2_s(weights=models.MViT_V2_S_Weights.DEFAULT).to(device)
model.eval()

# Transformaciones para MViT
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.45, 0.45, 0.45], std=[0.225, 0.225, 0.225])
])

# Función para extraer exactamente 16 frames de un video en un rango específico
def extract_frames(video_path, start_frame=60, end_frame=80, num_frames=16):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError(f"No se pudo abrir el video: {video_path}")
    
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames == 0:
        raise RuntimeError("El video no contiene frames")
    
    # Ajustar los límites del rango
    start_frame = max(0, min(start_frame, total_frames - 1))
    end_frame = max(start_frame, min(end_frame, total_frames - 1))
    frame_indices = np.linspace(start_frame, end_frame, num_frames, dtype=int)  # Seleccionar 16 frames equidistantes
    
    for i in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = Image.fromarray(frame)
        frames.append(transform(frame))
    
    cap.release()
    if len(frames) < num_frames:
        frames += [frames[-1]] * (num_frames - len(frames))  # Rellenar si hay menos de 16 frames
    
    frames_tensor = torch.stack(frames).permute(1, 0, 2, 3).unsqueeze(0).to(device)  # (1, 3, 16, 224, 224)
    return frames_tensor

# Función para extraer características con MViT
def extract_features(video_path, start_frame, end_frame):
    frames = extract_frames(video_path, start_frame, end_frame)
    #print("Frames tensor shape:", frames.shape)
    with torch.no_grad():
        features = model(frames)
    return features.cpu().numpy()

def extract_features_from_grouped_videos(grouped_videos, start_frame=60, end_frame=80):
    action_features = {}
    
    # Obtener la cantidad total de videos para la barra de progreso
    total_videos = sum(len(videos) for videos in grouped_videos.values())
    
    with tqdm(total=total_videos, desc="Procesando videos", unit="video") as pbar:
        for action, video_paths in grouped_videos.items():
            action_clips = []
            for video_path in video_paths:
                
                try:
                    features = extract_features(video_path, start_frame, end_frame)
                    action_clips.append(features)
                except Exception as e:
                    print(f"Error al procesar el video {video_path}: {e}")
                
                pbar.update(1)  # Actualizar la barra de progreso
            
            if action_clips:
                action_features[action] = np.mean(action_clips, axis=0)  # Promedio de características por acción
    
    return action_features


# Uso: extraer características y hacer mean por acción
videos_path = group_videos_by_action(root_path)
sorted_videos_path = ordenar_diccionario(videos_path)

# Extraer características para cada acción, ahora usando los videos del diccionario
features_dict = extract_features_from_grouped_videos(chunk, start_frame=60, end_frame=80)
print("Extracción y cálculo de media completados para todas las acciones.")




Procesando videos:   3%|█▊                                                        | 39/1251 [00:56<29:16,  1.45s/video]

In [18]:
# Guardar los resultados parciales
with open('train_augmented_result_1.pkl', 'wb') as f:
    pickle.dump(features_dict, f)  # Guarda el diccionario de resultados

In [15]:
# Cargar desde el archivo
with open('train_augmented_result_0.pkl', 'rb') as file:  # 'rb' es lectura en modo binario
    loaded_data = pickle.load(file)
  # Muestra el diccionario cargado

sample_keys = list(loaded_data.keys())  # Obtener las primeras 10 claves
sample_dict = {key for key in sample_keys}
print("Ejemplo de datos del diccionario:", sample_dict)

Ejemplo de datos del diccionario: {'action_3075', 'action_3255', 'action_3314', 'action_3135', 'action_3352', 'action_3409', 'action_3146', 'action_3068', 'action_3347', 'action_3300', 'action_3082', 'action_3260', 'action_3354', 'action_3291', 'action_3231', 'action_3348', 'action_3136', 'action_2958', 'action_2984', 'action_3156', 'action_3344', 'action_2980', 'action_2992', 'action_2959', 'action_3184', 'action_3346', 'action_3198', 'action_3244', 'action_2987', 'action_3297', 'action_3000', 'action_3234', 'action_3320', 'action_3029', 'action_3072', 'action_2943', 'action_3021', 'action_3364', 'action_3097', 'action_3337', 'action_3228', 'action_3275', 'action_3062', 'action_3280', 'action_3168', 'action_3009', 'action_3111', 'action_3194', 'action_3008', 'action_3399', 'action_3139', 'action_3290', 'action_3081', 'action_3202', 'action_3170', 'action_3252', 'action_3100', 'action_3037', 'action_3101', 'action_3249', 'action_3052', 'action_3140', 'action_3267', 'action_3032', 'acti

In [19]:
#Funciones para juntar los resultados parciales en un único archivo

directory_path = "F:/MVFoulRecognition/features/MVit/train/severity"
prefix = "train_"

files = []

for filename in os.listdir(directory_path):
    full_path = os.path.join(directory_path, filename)
    if os.path.isfile(full_path) and filename.startswith(prefix):
        files.append(full_path)

print("Archivos encontrados:")
sorted_files = sorted(files, key=lambda x: int(x.split('_result_')[-1].split('.pkl')[0]))

for file in sorted_files:
    print(file)
    
combined_data = []

# Cargar y combinar los datos
for file in sorted_files:
    with open(file, 'rb') as f:
        data = pickle.load(f)  # Carga el archivo .pkl
        combined_data.append(data)  # Agrega el array a la lista

# Guardar el array combinado en un nuevo archivo .pkl
with open('train_augmented.pkl', 'wb') as f:
    pickle.dump(combined_data, f)

print("Arrays combinados guardados en 'test_combined.pkl'")


Archivos encontrados:
F:/MVFoulRecognition/features/MVit/train/severity\train_augmented_result_0.pkl
F:/MVFoulRecognition/features/MVit/train/severity\train_augmented_result_1.pkl
Arrays combinados guardados en 'test_combined.pkl'


In [21]:
#Agrupamos en un mismo archivo los resultados de la extraccion de videos y videos aumentados

# Nombres de los archivos
archivo2 = "F:/MVFoulRecognition/features/MVit/train/severity/end/train_augmented.pkl"
archivo1 = "F:/MVFoulRecognition/features/MVit/train/severity/end/train_combined.pkl"
archivo_combinado = "train_combinados.pkl"

# Cargar los archivos
with open(archivo1, "rb") as f1, open(archivo2, "rb") as f2:
    datos1 = pickle.load(f1)
    datos2 = pickle.load(f2)

# Combinar los datos según el tipo de estructura
if isinstance(datos1, list) and isinstance(datos2, list):
    datos_combinados = datos1 + datos2  # Unir listas
elif isinstance(datos1, dict) and isinstance(datos2, dict):
    datos_combinados = {**datos1, **datos2}  # Unir diccionarios
else:
    raise TypeError("Los archivos tienen estructuras incompatibles")

# Guardar el archivo combinado
with open(archivo_combinado, "wb") as f:
    pickle.dump(datos_combinados, f)

print(f"Archivos combinados y guardados en '{archivo_combinado}'")


Archivos combinados y guardados en 'train_combinados.pkl'


# A partir de aquí termina la parte de extracción de features y comienza el entrenamiento (podemos mirar otro notebook)

In [76]:
with open('F:/MVFoulRecognition/features/Mvit/train/severity/end/train_combinados.pkl', 'rb') as file:  # 'rb' es lectura en modo binario
    loaded_data = pickle.load(file)

resultado_train = {k: v for dic in loaded_data for k, v in dic.items()}

# Salida: {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6}
dic_ordenado_train = dict(sorted(resultado_train.items(), key=lambda x: int(x[0].split('_')[1])))

#print(dic_ordenado_train)
with open('F:/MVFoulRecognition/features/Mvit/test/test_combined.pkl', 'rb') as file:  # 'rb' es lectura en modo binario
    loaded_data = pickle.load(file)

resultado_test = {k: v for dic in loaded_data for k, v in dic.items()}

# Salida: {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6}
dic_ordenado_test = dict(sorted(resultado_test.items(), key=lambda x: int(x[0].split('_')[1])))



In [77]:
def load_action_labels(json_path):
    """
    Carga las etiquetas de acción desde un archivo JSON.

    Args:
        json_path (str): Ruta al archivo JSON.

    Returns:
        dict: Diccionario donde las claves son nombres de acciones (ej. 'action_0')
              y los valores son las etiquetas (ej. 'Challenge', 'Tackling').
    """
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    action_labels = {}
    for action_id, action_data in data['Actions'].items():
        label = action_data['Severity']  # Extraer la etiqueta de 'Action class'
        action_name = f"action_{action_id}"  # Formar el nombre de la acción
        action_labels[action_name] = label
    
    return action_labels



In [78]:
ls = load_action_labels("F:/data/mvfouls/train/annotations.json")
ls = sorted(ls.items())
sorted_labels_train = sorted(ls, key=lambda x: int(x[0].split('_')[1]))


In [79]:
ls = load_action_labels("F:/data/mvfouls/test/annotations.json")
ls = sorted(ls.items())
sorted_labels_test = sorted(ls, key=lambda x: int(x[0].split('_')[1]))

In [80]:
#Comprobacion datos cargados
# Crear un diccionario con claves numéricas del 0 al 3999
diccionario = {i: 0 for i in range(0, 4052)}

# Recorrer las claves en `res`
for r in dic_ordenado_train:
    # Extraer el número de la clave 'action_X'
    action = int(r.split("_")[1])  # Convertimos el número extraído a entero
    
    # Si el número extraído de `action_X` está en `diccionario`, lo marcamos como 1
    if action in diccionario:
        diccionario[action] = 1

# Mostrar el diccionario actualizado
diccionario_filtrado = {k: v for k, v in diccionario.items() if v == 0}
print(diccionario_filtrado)

{3310: 0, 3691: 0, 3848: 0}


In [81]:
print(len(sorted_labels_train))
print(len(dic_ordenado_train))

2916
4050


CAMBIOS PARA SEVERITY

In [82]:
####AÑADIMOS NUEVAS ACCIONES AL DICCIONARIO DE LABELS 

# Suponiendo que tienes este diccionario de acciones con las etiquetas actuales en 'labels'
# Ejemplo:
# labels = [('action_0', 'Offence'), ('action_1', 'Offence'), ...]

# Calcular cuántas nuevas acciones necesitamos para llegar a 3345
new_actions_needed = 3174 - len(sorted_labels_train)+1

# Generar nuevas acciones con la etiqueta 'No Offence'
new_actions = [('action_' + str(i), '3.0') for i in range(len(sorted_labels_train), len(sorted_labels_train) + new_actions_needed)]

# Agregar las nuevas acciones al diccionario
updated_labels = sorted_labels_train + new_actions

# Verificar el total final y mostrar algunas de las nuevas acciones
print(f"Total de acciones después de la ampliación: {len(updated_labels)}")
print(updated_labels[:])  # Mostrar las últimas 10 acciones para verificar

# Calcular cuántas nuevas acciones necesitamos para llegar a 3345
new_actions_needed = 4052 - len(updated_labels)+1

# Crear las nuevas acciones con etiqueta '5.0' desde el último índice actual
start_index = int(updated_labels[-1][0].split('_')[1]) + 1  # último índice + 1

# Generar nuevas acciones
new_actions_5_0 = [('action_' + str(i), '5.0') for i in range(start_index, start_index + new_actions_needed)]

# Agregar al listado actualizado
updated_labels += new_actions_5_0

# Verificación
print(f"Total final de acciones: {len(updated_labels)}")
print("Últimas acciones añadidas:")
print(updated_labels[-new_actions_needed:])


Total de acciones después de la ampliación: 3175
[('action_0', '1.0'), ('action_1', '3.0'), ('action_2', '3.0'), ('action_3', '5.0'), ('action_4', '1.0'), ('action_5', '1.0'), ('action_6', ''), ('action_7', '1.0'), ('action_8', '1.0'), ('action_9', '1.0'), ('action_10', '1.0'), ('action_11', '1.0'), ('action_12', '1.0'), ('action_13', '1.0'), ('action_14', '3.0'), ('action_15', '1.0'), ('action_16', '1.0'), ('action_17', '1.0'), ('action_18', '3.0'), ('action_19', ''), ('action_20', '2.0'), ('action_21', '1.0'), ('action_22', '1.0'), ('action_23', '1.0'), ('action_24', '4.0'), ('action_25', ''), ('action_26', '1.0'), ('action_27', '3.0'), ('action_28', '1.0'), ('action_29', '3.0'), ('action_30', '2.0'), ('action_31', '3.0'), ('action_32', ''), ('action_33', ''), ('action_34', '1.0'), ('action_35', '2.0'), ('action_36', '1.0'), ('action_37', '3.0'), ('action_38', ''), ('action_39', '1.0'), ('action_40', ''), ('action_41', '3.0'), ('action_42', '3.0'), ('action_43', '1.0'), ('action_44',

In [83]:
import pandas as pd
from collections import Counter


# Convertir a DataFrame
df = pd.DataFrame(sorted_labels_train, columns=['action', 'label'])

# Contar la frecuencia de cada clase
class_counts = Counter(df['label'])

# Mostrar la distribución de clases
print("Distribución de clases:")
print(class_counts)

Distribución de clases:
Counter({'1.0': 1402, '3.0': 687, '2.0': 403, '': 353, '4.0': 44, '5.0': 27})


In [84]:
# Diccionario de mapeo
conversion = {
    "1.0": "no card",
    "2.0": "no card",
    "3.0": "yellow card",
    "4.0": "yellow card",
    "5.0": "red card",
    "": ""  # dejamos las vacías tal cual
}

# Aplicar conversión
updated_labels = [(action, conversion[label]) for action, label in updated_labels]

# Verificamos algunas etiquetas convertidas
print(sorted_labels_train[:10])




[('action_0', '1.0'), ('action_1', '3.0'), ('action_2', '3.0'), ('action_3', '5.0'), ('action_4', '1.0'), ('action_5', '1.0'), ('action_6', ''), ('action_7', '1.0'), ('action_8', '1.0'), ('action_9', '1.0')]


In [85]:
import pandas as pd
from collections import Counter


# Convertir a DataFrame
df = pd.DataFrame(updated_labels, columns=['action', 'label'])

# Contar la frecuencia de cada clase
class_counts = Counter(df['label'])

# Mostrar la distribución de clases
print("Distribución de clases:")
print(class_counts)

Distribución de clases:
Counter({'no card': 1805, 'yellow card': 990, 'red card': 905, '': 353})


In [86]:
# 1. Filtrar las acciones con etiqueta 'Offence'
offence_actions = [item for item in updated_labels if item[1] == ""]

# 2. Seleccionar las primeras mil acciones con etiqueta '""'
# Aseguramos que hay al menos mil acciones con etiqueta '""'
actions_to_remove = offence_actions

# 3. Guardar estas mil acciones en una lista
actions_to_remove_list = actions_to_remove

updated_labels = [item for item in updated_labels if item not in actions_to_remove_list]

for action in actions_to_remove_list:
    dic_ordenado_train.pop(action[0], None) 
# Ver el diccionario después de la eliminación
print("Diccionario después de eliminar las acciones:", dic_ordenado_train.keys())

Diccionario después de eliminar las acciones: dict_keys(['action_0', 'action_1', 'action_2', 'action_3', 'action_4', 'action_5', 'action_7', 'action_8', 'action_9', 'action_10', 'action_11', 'action_12', 'action_13', 'action_14', 'action_15', 'action_16', 'action_17', 'action_18', 'action_20', 'action_21', 'action_22', 'action_23', 'action_24', 'action_26', 'action_27', 'action_28', 'action_29', 'action_30', 'action_31', 'action_34', 'action_35', 'action_36', 'action_37', 'action_39', 'action_41', 'action_42', 'action_43', 'action_44', 'action_45', 'action_46', 'action_47', 'action_48', 'action_49', 'action_50', 'action_51', 'action_52', 'action_53', 'action_54', 'action_55', 'action_56', 'action_57', 'action_59', 'action_60', 'action_61', 'action_62', 'action_64', 'action_65', 'action_66', 'action_67', 'action_68', 'action_69', 'action_70', 'action_71', 'action_72', 'action_73', 'action_74', 'action_75', 'action_76', 'action_77', 'action_78', 'action_79', 'action_80', 'action_81', 'ac

In [87]:
import pandas as pd
from collections import Counter


# Convertir a DataFrame
df = pd.DataFrame(updated_labels, columns=['action', 'label'])

# Contar la frecuencia de cada clase
class_counts = Counter(df['label'])

# Mostrar la distribución de clases
print("Distribución de clases:")
print(class_counts)

Distribución de clases:
Counter({'no card': 1805, 'yellow card': 990, 'red card': 905})


In [88]:
# 1. Filtrar las acciones con etiqueta 'Offence'
offence_actions = [item for item in updated_labels if item[1] == 'no card']

# 2. Seleccionar las primeras mil acciones con etiqueta 'Offence'
# Aseguramos que hay al menos mil acciones con etiqueta 'Offence'
actions_to_remove = offence_actions[-500:]

# 3. Guardar estas mil acciones en una lista
actions_to_remove_list = actions_to_remove

updated_labels = [item for item in updated_labels if item not in actions_to_remove_list]

for action in actions_to_remove_list:
    dic_ordenado_train.pop(action[0], None) 
# Ver el diccionario después de la eliminación
print("Diccionario después de eliminar las acciones:", dic_ordenado_train.keys())

Diccionario después de eliminar las acciones: dict_keys(['action_0', 'action_1', 'action_2', 'action_3', 'action_4', 'action_5', 'action_7', 'action_8', 'action_9', 'action_10', 'action_11', 'action_12', 'action_13', 'action_14', 'action_15', 'action_16', 'action_17', 'action_18', 'action_20', 'action_21', 'action_22', 'action_23', 'action_24', 'action_26', 'action_27', 'action_28', 'action_29', 'action_30', 'action_31', 'action_34', 'action_35', 'action_36', 'action_37', 'action_39', 'action_41', 'action_42', 'action_43', 'action_44', 'action_45', 'action_46', 'action_47', 'action_48', 'action_49', 'action_50', 'action_51', 'action_52', 'action_53', 'action_54', 'action_55', 'action_56', 'action_57', 'action_59', 'action_60', 'action_61', 'action_62', 'action_64', 'action_65', 'action_66', 'action_67', 'action_68', 'action_69', 'action_70', 'action_71', 'action_72', 'action_73', 'action_74', 'action_75', 'action_76', 'action_77', 'action_78', 'action_79', 'action_80', 'action_81', 'ac

In [89]:
import pandas as pd
from collections import Counter


# Convertir a DataFrame
df = pd.DataFrame(updated_labels, columns=['action', 'label'])

# Contar la frecuencia de cada clase
class_counts = Counter(df['label'])

# Mostrar la distribución de clases
print("Distribución de clases:")
print(class_counts)

Distribución de clases:
Counter({'no card': 1305, 'yellow card': 990, 'red card': 905})


In [90]:
import pandas as pd
from collections import Counter


# Convertir a DataFrame
df = pd.DataFrame(sorted_labels_test, columns=['action', 'label'])

# Contar la frecuencia de cada clase
class_counts = Counter(df['label'])

# Mostrar la distribución de clases
print("Distribución de clases:")
print(class_counts)

Distribución de clases:
Counter({'1.0': 168, '3.0': 68, '2.0': 30, '': 26, '5.0': 5, '4.0': 4})


In [91]:
# Diccionario de mapeo
conversion = {
    "1.0": "no card",
    "2.0": "no card",
    "3.0": "yellow card",
    "4.0": "yellow card",
    "5.0": "red card",
    "": ""  # dejamos las vacías tal cual
}

# Aplicar conversión
sorted_labels_test = [(action, conversion[label]) for action, label in sorted_labels_test]

# Verificamos algunas etiquetas convertidas
print(sorted_labels_test[:10])




[('action_0', 'no card'), ('action_1', 'yellow card'), ('action_2', 'no card'), ('action_3', 'no card'), ('action_4', 'no card'), ('action_5', 'yellow card'), ('action_6', 'no card'), ('action_7', 'no card'), ('action_8', 'yellow card'), ('action_9', 'no card')]


In [92]:
# 1. Filtrar las acciones con etiqueta 'Offence'
offence_actions = [item for item in sorted_labels_test if item[1] == ""]

# 2. Seleccionar las primeras mil acciones con etiqueta '""'
# Aseguramos que hay al menos mil acciones con etiqueta '""'
actions_to_remove = offence_actions

# 3. Guardar estas mil acciones en una lista
actions_to_remove_list = actions_to_remove

sorted_labels_test = [item for item in sorted_labels_test if item not in actions_to_remove_list]

for action in actions_to_remove_list:
    dic_ordenado_test.pop(action[0], None) 
# Ver el diccionario después de la eliminación
print("Diccionario después de eliminar las acciones:", dic_ordenado_test.keys())

Diccionario después de eliminar las acciones: dict_keys(['action_0', 'action_1', 'action_2', 'action_3', 'action_4', 'action_5', 'action_6', 'action_7', 'action_8', 'action_9', 'action_10', 'action_11', 'action_12', 'action_13', 'action_14', 'action_15', 'action_16', 'action_17', 'action_18', 'action_19', 'action_20', 'action_21', 'action_22', 'action_23', 'action_24', 'action_25', 'action_26', 'action_27', 'action_28', 'action_29', 'action_30', 'action_31', 'action_32', 'action_33', 'action_34', 'action_35', 'action_36', 'action_37', 'action_38', 'action_39', 'action_40', 'action_42', 'action_43', 'action_44', 'action_45', 'action_46', 'action_48', 'action_49', 'action_50', 'action_51', 'action_52', 'action_53', 'action_54', 'action_55', 'action_56', 'action_57', 'action_58', 'action_60', 'action_61', 'action_62', 'action_64', 'action_66', 'action_67', 'action_68', 'action_70', 'action_71', 'action_72', 'action_73', 'action_74', 'action_75', 'action_76', 'action_77', 'action_79', 'act

In [93]:
updated_labels = [item for item in updated_labels if item[0] != 'action_3320' and item[0] != 'action_3691' and item[0] != 'action_3848' ]


In [316]:
labels_dic_train.keys()

dict_keys(['action_1', 'action_2', 'action_14', 'action_18', 'action_24', 'action_27', 'action_29', 'action_31', 'action_37', 'action_41', 'action_42', 'action_44', 'action_46', 'action_50', 'action_51', 'action_57', 'action_70', 'action_74', 'action_77', 'action_78', 'action_81', 'action_87', 'action_96', 'action_97', 'action_99', 'action_101', 'action_102', 'action_105', 'action_108', 'action_109', 'action_110', 'action_116', 'action_118', 'action_120', 'action_123', 'action_124', 'action_126', 'action_131', 'action_134', 'action_136', 'action_137', 'action_138', 'action_139', 'action_140', 'action_143', 'action_147', 'action_148', 'action_149', 'action_150', 'action_157', 'action_159', 'action_164', 'action_165', 'action_169', 'action_170', 'action_174', 'action_188', 'action_192', 'action_194', 'action_196', 'action_199', 'action_200', 'action_202', 'action_205', 'action_207', 'action_212', 'action_213', 'action_215', 'action_219', 'action_227', 'action_229', 'action_235', 'action_

In [317]:
dic_ordenado_train.keys()

dict_keys(['action_1', 'action_2', 'action_3', 'action_14', 'action_18', 'action_24', 'action_27', 'action_29', 'action_31', 'action_37', 'action_41', 'action_42', 'action_44', 'action_46', 'action_50', 'action_51', 'action_57', 'action_70', 'action_74', 'action_77', 'action_78', 'action_81', 'action_82', 'action_87', 'action_96', 'action_97', 'action_99', 'action_101', 'action_102', 'action_105', 'action_108', 'action_109', 'action_110', 'action_116', 'action_118', 'action_120', 'action_123', 'action_124', 'action_126', 'action_128', 'action_131', 'action_134', 'action_136', 'action_137', 'action_138', 'action_139', 'action_140', 'action_143', 'action_147', 'action_148', 'action_149', 'action_150', 'action_154', 'action_157', 'action_159', 'action_164', 'action_165', 'action_169', 'action_170', 'action_174', 'action_188', 'action_192', 'action_194', 'action_196', 'action_199', 'action_200', 'action_202', 'action_205', 'action_207', 'action_209', 'action_212', 'action_213', 'action_215

In [94]:
X_train  =np.array(list(dic_ordenado_train.values()))
labels_dic_train = dict(updated_labels)
Y_train = np.array(list(labels_dic_train.values()))

X_test = np.array(list(dic_ordenado_test.values()))
labels_dict_test = dict(sorted_labels_test)
Y_test = np.array(list(labels_dict_test.values()))

print("Ejemplo de etiquetas en Y_train:", Y_train[:10])
print("Ejemplo de etiquetas en Y_valid:", Y_test[:10])

label_encoder = LabelEncoder()

# Ajustamos el codificador con TODAS las etiquetas posibles
label_encoder.fit(np.concatenate([Y_train, Y_test]))

# Transformamos las etiquetas de texto a números
Y_train_encoded = label_encoder.transform(Y_train)
Y_test_encoded = label_encoder.transform(Y_test)

print("Etiquetas originales:", np.unique(Y_train)) 
print("Etiquetas codificadas:", np.unique(Y_train_encoded))
print("Diccionario de conversión:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))


Ejemplo de etiquetas en Y_train: ['no card' 'yellow card' 'yellow card' 'red card' 'no card' 'no card'
 'no card' 'no card' 'no card' 'no card']
Ejemplo de etiquetas en Y_valid: ['no card' 'yellow card' 'no card' 'no card' 'no card' 'yellow card'
 'no card' 'no card' 'yellow card' 'no card']
Etiquetas originales: ['no card' 'red card' 'yellow card']
Etiquetas codificadas: [0 1 2]
Diccionario de conversión: {np.str_('no card'): np.int64(0), np.str_('red card'): np.int64(1), np.str_('yellow card'): np.int64(2)}


In [95]:
print(f"X_train type: {type(X_train)}, Y_train type: {type(Y_train)}")
print(f"X_valid type: {type(X_test)}, Y_valid type: {type(Y_test)}")

# Verificar las dimensiones
print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_valid shape: {X_test.shape}, Y_valid shape: {Y_test.shape}")




X_train type: <class 'numpy.ndarray'>, Y_train type: <class 'numpy.ndarray'>
X_valid type: <class 'numpy.ndarray'>, Y_valid type: <class 'numpy.ndarray'>
X_train shape: (3197, 1, 400), Y_train shape: (3197,)
X_valid shape: (275, 1, 400), Y_valid shape: (275,)


In [96]:
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from scipy.stats import uniform
import numpy as np
from sklearn.model_selection import train_test_split

# Aplanar la segunda dimensión de X_train y X_valid
X_train_flattened = X_train.squeeze(1)  # (2916, 400)
X_test_flattened = X_test.squeeze(1)  # (301, 400)
# Suponiendo que estos son tus datos completos
X_total = np.concatenate((X_train_flattened, X_test_flattened))
Y_total = np.concatenate((Y_train_encoded, Y_test_encoded))

# Nueva división estratificada
X_train, X_test, Y_train, Y_test = train_test_split(
    X_total, Y_total,
    test_size=0.2,         # o el porcentaje que quieras
    stratify=Y_total,      # 👈 esto asegura la misma proporción de clases
    random_state=42
)

# Definir el espacio de búsqueda de hiperparámetros
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],  # Número de árboles
    'learning_rate': uniform(0.01, 0.2),         # Tasa de aprendizaje
    'max_depth': [3, 4, 5, 6, 7],                # Profundidad máxima de los árboles
    'min_child_weight': [1, 2, 3, 4],            # Peso mínimo de los hijos
    'subsample': uniform(0.6, 0.4),              # Submuestra de las instancias
    'colsample_bytree': uniform(0.6, 0.4),       # Submuestra de las columnas
    'gamma': [0, 0.1, 0.2, 0.3],                 # Penalización por complejidad
    'class_weight': ['balanced', None],          # Manejo del desbalance
}

# Definir el modelo XGBoost
xgb = XGBClassifier(random_state=42)

# Realizar Randomized Search
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=30,  # Número de combinaciones a probar
    scoring='f1_macro',  # Usar F1 macro como métrica de optimización
    cv=3,  # Validación cruzada
    random_state=42,
    verbose=2,
    n_jobs=-1  # Para usar todos los núcleos del procesador
)

# Ajustar al conjunto de entrenamiento
random_search.fit(X_train, Y_train)

# Resultados de los mejores parámetros
print(f"Mejores parámetros encontrados: {random_search.best_params_}")

# Usar el mejor modelo
best_xgb = random_search.best_estimator_

# Realizar predicciones en el conjunto de prueba
Y_test_pred = best_xgb.predict(X_test)

# Evaluar el modelo
print(f"Precisión del modelo optimizado: {accuracy_score(Y_test, Y_test_pred)}")
print("Matriz de Confusión:")
print(confusion_matrix(Y_test, Y_test_pred))
print("Reporte de Clasificación:")
print(classification_report(Y_test, Y_test_pred))


Fitting 3 folds for each of 30 candidates, totalling 90 fits
Mejores parámetros encontrados: {'class_weight': None, 'colsample_bytree': np.float64(0.815876852955632), 'gamma': 0.3, 'learning_rate': np.float64(0.08135066533871785), 'max_depth': 6, 'min_child_weight': 4, 'n_estimators': 400, 'subsample': np.float64(0.7103996728090174)}
Precisión del modelo optimizado: 0.7381294964028777
Matriz de Confusión:
[[260   1  40]
 [  9 162  10]
 [111  11  91]]
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.68      0.86      0.76       301
           1       0.93      0.90      0.91       181
           2       0.65      0.43      0.51       213

    accuracy                           0.74       695
   macro avg       0.75      0.73      0.73       695
weighted avg       0.74      0.74      0.73       695



In [39]:
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from scipy.stats import uniform
import numpy as np
from sklearn.model_selection import train_test_split

# Aplanar la segunda dimensión de X_train y X_valid
X_train_flattened = X_train.squeeze(1)  # (2916, 400)
X_test_flattened = X_test.squeeze(1)  # (301, 400)

# Suponiendo que estos son tus datos completos
X_total = np.concatenate((X_train_flattened, X_test_flattened))
Y_total = np.concatenate((Y_train_encoded, Y_test_encoded))

# Nueva división estratificada
X_train, X_test, Y_train, Y_test = train_test_split(
    X_total, Y_total,
    test_size=0.2,  # o el porcentaje que quieras
    stratify=Y_total,  # 👈 esto asegura la misma proporción de clases
    random_state=42
)

# Aplicar SMOTE para balancear las clases en el conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_resampled, Y_train_resampled = smote.fit_resample(X_train, Y_train)

# Definir el espacio de búsqueda de hiperparámetros
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],  # Número de árboles
    'learning_rate': uniform(0.01, 0.2),         # Tasa de aprendizaje
    'max_depth': [3, 4, 5, 6, 7],                # Profundidad máxima de los árboles
    'min_child_weight': [1, 2, 3, 4],            # Peso mínimo de los hijos
    'subsample': uniform(0.6, 0.4),              # Submuestra de las instancias
    'colsample_bytree': uniform(0.6, 0.4),       # Submuestra de las columnas
    'gamma': [0, 0.1, 0.2, 0.3],                 # Penalización por complejidad
    'class_weight': ['balanced', None],          # Manejo del desbalance
}

# Definir el modelo XGBoost
xgb = XGBClassifier(random_state=42)

# Realizar Randomized Search
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=30,  # Número de combinaciones a probar
    scoring='recall_weighted',  # Usar F1 macro como métrica de optimización
    cv=3,  # Validación cruzada
    random_state=42,
    verbose=2,
    n_jobs=-1  # Para usar todos los núcleos del procesador
)

# Ajustar al conjunto de entrenamiento con las muestras balanceadas
random_search.fit(X_train_resampled, Y_train_resampled)

# Resultados de los mejores parámetros
print(f"Mejores parámetros encontrados: {random_search.best_params_}")

# Usar el mejor modelo
best_xgb = random_search.best_estimator_

# Realizar predicciones en el conjunto de prueba
Y_test_pred = best_xgb.predict(X_test)

# Evaluar el modelo
print(f"Precisión del modelo optimizado: {accuracy_score(Y_test, Y_test_pred)}")
print("Matriz de Confusión:")
print(confusion_matrix(Y_test, Y_test_pred))
print("Reporte de Clasificación:")
print(classification_report(Y_test, Y_test_pred))


Fitting 3 folds for each of 30 candidates, totalling 90 fits
Mejores parámetros encontrados: {'class_weight': None, 'colsample_bytree': np.float64(0.9849789179768444), 'gamma': 0.1, 'learning_rate': np.float64(0.14926085456795768), 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 500, 'subsample': np.float64(0.713936197750987)}
Precisión del modelo optimizado: 0.7235772357723578
Matriz de Confusión:
[[159   2  60]
 [  3 174   4]
 [ 91  10 112]]
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.63      0.72      0.67       221
           1       0.94      0.96      0.95       181
           2       0.64      0.53      0.58       213

    accuracy                           0.72       615
   macro avg       0.73      0.74      0.73       615
weighted avg       0.72      0.72      0.72       615



In [249]:
from sklearn.metrics import classification_report, confusion_matrix

print("Matriz de Confusión:")

print(confusion_matrix(Y_test_encoded, Y_test_pred))

print("\nReporte de Clasificación:")
print(classification_report(Y_test_encoded, Y_test_pred))

Matriz de Confusión:


ValueError: Found input variables with inconsistent numbers of samples: [275, 572]

In [61]:
import pickle

# Guarda el modelo en un archivo
with open('modelo_xgboost_severity.pkl', 'wb') as f:
    pickle.dump(best_xgb, f)


In [ ]:
import pickle

# Cargar el modelo desde el archivo
with open("modelo_xgboost.pkl", "rb") as archivo:
    modelo = pickle.load(archivo)


In [ ]:
modelo